In [6]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from scipy.sparse import lil_matrix
import ast
df = pd.read_csv("data/clean_data/recipes_cleaned.csv")
df.head()

,name_x,nutrition,n_steps,ingredients_y,minutes
0,arriba baked winter squash mexican style,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['winter squash', 'mexican seasoning', 'mixed ...",55
1,a bit different breakfast pizza,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['prepared pizza crust', 'sausage patty', 'egg...",30
2,all in the kitchen chili,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['ground beef', 'yellow onions', 'diced tomato...",130
3,alouette potatoes,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,"['spreadable cheese with garlic and herbs', 'n...",45
4,amish tomato ketchup for canning,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"['tomato juice', 'apple cider vinegar', 'sugar...",190


In [7]:
# Parser la colonne ingredients_y
# Vérifier si c'est une string ou déjà une liste
print(f"\nType de ingredients_y : {type(df['ingredients_y'].iloc[0])}")
print(f"Exemple : {df['ingredients_y'].iloc[0]}")

# Si c'est une string (ex: "['salt', 'pepper']"), parser en liste
if isinstance(df['ingredients_y'].iloc[0], str):
    print("Parsing ingredients_y de string vers list...")
    df['ingredients_y'] = df['ingredients_y'].apply(ast.literal_eval)


Type de ingredients_y : <class 'str'>
Exemple : ['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
Parsing ingredients_y de string vers list...


In [8]:
# Construire le vocabulaire d'ingrédients
all_ingredients = []
for ing_list in df['ingredients_y']:
    # Dédupliquer dans chaque recette
    all_ingredients.extend(set(ing_list))

unique_ingredients = sorted(set(all_ingredients))
ingredients_index = {ing: idx for idx, ing in enumerate(unique_ingredients)}
n = len(unique_ingredients)

print(f"  Nombre de recettes : {len(df)}")
print(f"  Ingrédients uniques : {n}")

  Nombre de recettes : 222705
  Ingrédients uniques : 14621


In [9]:
# Vérifier la distribution 
ing_per_recipe = df['ingredients_y'].apply(lambda x: len(set(x)))
print(f"  Ingrédients/recette : min={ing_per_recipe.min()}, "
      f"max={ing_per_recipe.max()}, moyenne={ing_per_recipe.mean():.1f}")

  Ingrédients/recette : min=1, max=43, moyenne=9.1


In [10]:
# Construire matrice binaire recettes × ingrédients
# Cette méthode est la plus robuste
n_recipes = len(df)
X = lil_matrix((n_recipes, n), dtype=bool)

for r, ing_list in enumerate(df['ingredients_y']):
    unique_in_recipe = set(ing_list)  # dédupliquer
    for ing in unique_in_recipe:
        X[r, ingredients_index[ing]] = True

# Convertir en CSR pour calculs efficaces
X = X.tocsr()

print(f"  Matrice X : {X.shape}, densité={X.nnz / (X.shape[0] * X.shape[1]):.4f}")

  Matrice X : (222705, 14621), densité=0.0006


In [11]:
# Calculer fréquences et vérifier cohérence 
deg = np.array(X.sum(axis=0)).flatten()  # somme par colonne = fréquence ingrédient

print(f"\n Fréquences des ingrédients :")
print(f"  Min : {deg.min()}")
print(f"  Max : {deg.max()}")
print(f"  Médiane : {np.median(deg):.0f}")
print(f"  Total : {deg.sum()}")

# Top 10 ingrédients
top10_idx = np.argsort(deg)[-10:][::-1]
print(f"\n Top 10 ingrédients les plus fréquents :")
for idx in top10_idx:
    print(f"  {unique_ingredients[idx]}: {deg[idx]} recettes")



 Fréquences des ingrédients :
  Min : 1
  Max : 83152
  Médiane : 5
  Total : 2028585

 Top 10 ingrédients les plus fréquents :
  salt: 83152 recettes
  butter: 53656 recettes
  sugar: 42544 recettes
  onion: 38174 recettes
  water: 33416 recettes
  eggs: 33226 recettes
  olive oil: 32138 recettes
  flour: 25676 recettes
  garlic cloves: 25147 recettes
  milk: 24854 recettes


In [ ]:
# --- ÉTAPE 7 (version rapide avec Cosine Distance) ---
from sklearn.metrics.pairwise import cosine_distances
import time, numpy as np

print("\n Calcul de la matrice de distances Cosine (approximation Jaccard)...")

start = time.time()

X_ingredients = X.T  # ingrédients × recettes
distance_matrix = cosine_distances(X_ingredients)  # beaucoup plus rapide

print(f"  Matrice de distances : {distance_matrix.shape}")
print(f"  Min distance : {distance_matrix.min():.4f}")
print(f"  Max distance : {distance_matrix.max():.4f}")
print(f"  Symétrique : {np.allclose(distance_matrix, distance_matrix.T)}")
print(f"✅ Calcul terminé en {(time.time()-start)/60:.2f} min")

np.save("data/distance_matrix_cosine.npy", distance_matrix)


In [ ]:
# Calculer distance de Jaccard (optimisée) 

from sklearn.metrics import pairwise_distances
from collections import Counter
import numpy as np

# Sélectionner les ingrédients les plus fréquents 
min_occurrences = 100   # à ajuster selon la taille mémoire disponible
ingredient_counts = Counter([ing for lst in df['ingredients_y'] for ing in lst])

selected_ingredients = {ing for ing, c in ingredient_counts.items() if c >= min_occurrences}
print(f"  Ingrédients sélectionnés (>= {min_occurrences} recettes) : {len(selected_ingredients)}")

# Filtrer les recettes pour ne garder que ces ingrédients
df['ingredients_filtered'] = df['ingredients_y'].apply(lambda lst: [i for i in lst if i in selected_ingredients])

# Reconstruire la matrice binaire (recette × ingrédient)
from sklearn.feature_extraction.text import CountVectorizer

corpus = [" ".join(ings) for ings in df['ingredients_filtered']]
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(corpus)

print(f"  Nouvelle matrice X : {X.shape} (recettes × ingrédients fréquents)")

# Calculer distances Jaccard (en espace ingrédients) 
# NB : Transposer la matrice pour comparer les ingrédients entre eux
print("  Calcul de la matrice de distances Jaccard...")

# Utilisation directe sur sparse matrix (plus sûr)
X_ingredients = X.T  # shape: (n_ingredients, n_recettes)

# Convertir en dense car Jaccard ne supporte pas sparse ici
distance_matrix = pairwise_distances(X_ingredients.toarray(), metric='jaccard', n_jobs=-1)

print(f"  Matrice de distances : {distance_matrix.shape}")
print(f"  Min distance : {distance_matrix.min():.4f}")
print(f"  Max distance : {distance_matrix.max():.4f}")
print(f"  Symétrique : {np.allclose(distance_matrix, distance_matrix.T)}")

# Vérifications
assert distance_matrix.min() >= 0, "Distances négatives!"
assert distance_matrix.max() <= 1, "Distances > 1!"
assert np.allclose(np.diag(distance_matrix), 0), "Diagonale non nulle!"

print(" Matrice de distances Jaccard valide et calculée avec succès !")


  Ingrédients sélectionnés (>= 100 recettes) : 1803
  Nouvelle matrice X : (222705, 1010) (recettes × ingrédients fréquents)
  Calcul de la matrice de distances Jaccard...


/Users/duybeobn1/miniconda/lib/python3.13/site-packages/sklearn/metrics/pairwise.py:2462: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [ ]:
# Choisir eps via k-distance plot 
from sklearn.neighbors import NearestNeighbors

k = 5  # min_samples
nbrs = NearestNeighbors(n_neighbors=k+1, metric='precomputed').fit(distance_matrix)
distances, indices = nbrs.kneighbors(distance_matrix)
kdist = np.sort(distances[:, k])  # k-ième voisin (exclut soi-même)

# Plot
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(kdist, linewidth=1)
plt.xlabel('Ingrédients (triés par k-distance)')
plt.ylabel(f'{k}-distance')
plt.title(f'K-distance plot pour déterminer eps (k={k})')
plt.grid(True, alpha=0.3)

# Suggérer eps au 90e percentile comme départ
eps_90 = np.percentile(kdist, 90)
eps_95 = np.percentile(kdist, 95)
plt.axhline(y=eps_90, color='orange', linestyle='--', label=f'90e percentile ({eps_90:.3f})')
plt.axhline(y=eps_95, color='red', linestyle='--', label=f'95e percentile ({eps_95:.3f})')
plt.legend()
plt.tight_layout()
plt.savefig('kdistance_plot.png', dpi=150)
plt.show()

print(f"\n Suggestions eps :")
print(f"  90e percentile : {eps_90:.3f}")
print(f"  95e percentile : {eps_95:.3f}")

In [ ]:
# DBSCAN
eps = eps_90  # ou ajuster manuellement selon le plot
min_samples = k

print(f"\n🎯 Exécution DBSCAN (eps={eps:.3f}, min_samples={min_samples})...")
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
labels = dbscan.fit_predict(distance_matrix)



In [ ]:
# Analyser 
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = (labels == -1).sum()

print(f"\n Résultats DBSCAN :")
print(f"  Nombre de clusters : {n_clusters}")
print(f"  Points de bruit (-1) : {n_noise} / {n} ({100*n_noise/n:.1f}%)")

# Distribution des tailles de clusters
cluster_sizes = {}
for label in set(labels):
    if label != -1:
        cluster_sizes[label] = (labels == label).sum()

if cluster_sizes:
    print(f"\n Taille des clusters :")
    for label in sorted(cluster_sizes.keys(), key=lambda x: cluster_sizes[x], reverse=True)[:10]:
        size = cluster_sizes[label]
        members = [unique_ingredients[i] for i, lbl in enumerate(labels) if lbl == label]
        print(f"  Cluster {label} : {size} ingrédients")
        print(f"    Exemples : {', '.join(members[:8])}")

# Afficher quelques ingrédients du bruit
if n_noise > 0:
    noise_ingredients = [unique_ingredients[i] for i, lbl in enumerate(labels) if lbl == -1]
    print(f"\n Exemples d'ingrédients isolés (bruit) :")
    print(f"  {', '.join(noise_ingredients[:15])}")


In [ ]:

# Sauvegarder
results_df = pd.DataFrame({
    'ingredient': unique_ingredients,
    'cluster': labels,
    'frequency': deg
})
results_df.to_csv('clean_data/ingredient_clusters.csv', index=False)
print(f"\n Résultats sauvegardés dans 'clean_data/ingredient_clusters.csv'")